In [1]:
using JuMP, Gurobi
using Random
using Plots

In [2]:
function generate_distance_matrix(n; random_seed = 1)
    rng = Random.MersenneTwister(random_seed)
    X = 100 * rand(rng, n) # x 좌표
    Y = 100 * rand(rng, n) # y 좌표
    d = [sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2) for i in 1:n, j in 1:n] #distance
    return X, Y, d
end

n = 10
X, Y, d = generate_distance_matrix(n)

([23.603334566204694, 34.651701419196044, 31.27069683360675, 0.790928339056074, 48.86128300795012, 21.096820215853597, 95.1916339835734, 99.99046588986135, 25.166218303197184, 98.66663668987997], [55.57510873245724, 43.71079746096251, 42.471785049513144, 77.3223048457377, 28.11902322857298, 20.947237319807076, 25.137920979222493, 2.037486871266725, 28.77015122756894, 85.9512136087661], [0.0 16.21197989343667 … 26.850481418667766 80.97658348662696; 16.21197989343667 0.0 … 17.697381156976782 76.69527165379544; … ; 26.850481418667766 17.697381156976782 … 0.0 93.12349541366977; 80.97658348662696 76.69527165379544 … 93.12349541366977 0.0])

In [5]:
function build_tsp_model(d, n)
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:n, 1:n], Bin, Symmetric)
    @objective(model, Min, sum(d .* x) / 2)
    @constraint(model, [i in 1:n], sum(x[i, :]) == 2)
    @constraint(model, [i in 1:n], x[i, i] == 0)
    return model
end

build_tsp_model (generic function with 1 method)

In [6]:
iterative_model = build_tsp_model(d, n)
optimize!(iterative_model)
time_iterated = solve_time(iterative_model)
#cycle = subtour(iterative_model[:x])
#while 1 < length(cycle) < n
    #println("Found cycle of length $(length(cycle))")
    #S = [(i, j) for (i, j) in Iterators.product(cycle, cycle) if i < j]
    #@constraint(
    #    iterative_model,
    #    sum(iterative_model[:x][i, j] for (i, j) in S) <= length(cycle) - 1,
    #)
    optimize!(iterative_model)
    global time_iterated += solve_time(iterative_model)
    #global cycle = subtour(iterative_model[:x])
end

objective_value(iterative_model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 20 rows, 55 columns and 110 nonzeros
Model fingerprint: 0x414e3370
Variable types: 0 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 439.4661225
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolved: 10 rows, 45 columns, 90 nonzeros
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: objective 3.272571e+02, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

LoadError: syntax: unexpected "end"

In [10]:
iterative_model = build_tsp_model(d, n)
optimize!(iterative_model)
time_iterated = solve_time(iterative_model)

# Check for fractional solutions and add Gomory cuts
fractional_solutions = [(i, j) for i in 1:n, j in 1:n if abs(value(x[i, j]) - round(value(x[i, j]))) > 1e-6]
if length(fractional_solutions) > 0
    println("\nFractional Solutions Found, Adding Gomory Cuts:")
    for (i, j) in fractional_solutions
        @constraint(model, x[i, j] == 0)
        optimize!(model)
        println("Cut added for x[$i, $j]")
    end
end

# Solve the MILP problem
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 20 rows, 55 columns and 110 nonzeros
Model fingerprint: 0x414e3370
Variable types: 0 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 439.4661225
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolved: 10 rows, 45 columns, 90 nonzeros
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: objective 3.272571e+02, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

LoadError: UndefVarError: `x` not defined

In [9]:
function selected_edges(x::Matrix{Float64}, n)
    return Tuple{Int,Int}[(i, j) for i in 1:n, j in 1:n if x[i, j] > 0.5]
end

selected_edges (generic function with 1 method)

In [1]:
using Pkg
Pkg.add("TrafficAssignment")

using TrafficAssignment

# 네트워크 정보
i_nodes = [1, 2, 2, 4]
t_nodes = [2, 3, 4, 3]
capacities = [100.0, 50.0, 100.0, 100.0]
free_flow_times = [10.0, 10.0, 100.0, 100.0]
ds_balances = [0.0, 50.0, 50.0, 50.0]

l_lengths = [0.0, 50.0, 50.0, 50.0]
power_factors = [1.0, 1.0, 1.0, 1.0]
speed_limits = [100.0, 100.0, 100.0, 100.0]
toll_costs = [0.0, 0.0, 0.0, 0.0]
l_types = [1, 1, 1, 1]

# 수요 정보
travel_demands = zeros(4, 4)
travel_demands[1, 3] = 60
travel_demands[1, 4] = 40
origin_destination_pairs = [(1, 3), (1, 4)]

# TA_Data 인스턴스
ta_data = TA_Data(
    "Problem1",  # network_name
    3,  # number_of_zones
    4,  # number_of_nodes
    1,  # first_thru_node
    4,  # number_of_links
    i_nodes,
    t_nodes,
    capacities,
    l_lengths,
    free_flow_times,
    ds_balances,
    power_factors,
    speed_limits,
    toll_costs,
    l_types,
    sum(travel_demands),  # total_od_flow
    travel_demands,
    origin_destination_pairs,
    0.5,  # toll_factor
    0.5,  # distance_factor
    Inf  # best_objective
)

link_flow, link_travel_time, objective =
ta_frank_wolfe(ta_data, method=:cfw, max_iter_no=1000, step=:newton, log=:on, tol=1e-5)

print("Link flow: ", link_flow, "\n")
print("Link travel time: ", link_travel_time, "\n")
print("Objective: ", objective, "\n")

using LinearAlgebra
system_travel_time = dot(link_travel_time, link_flow)

print("System travel time: ", system_travel_time, "\n")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Calculus ─────────────── v0.5.1
   Installed JpegTurbo_jll ────────── v3.0.1+0
   Installed URIParser ────────────── v0.4.1
   Installed PlotUtils ────────────── v1.2.0
   Installed InvertedIndices ──────── v1.3.0
   Installed Unitful ──────────────── v1.19.0
   Installed MutableArithmetics ───── v1.4.0
   Installed StaticArrays ─────────── v0.12.6
   Installed HTTP ─────────────────── v1.10.1
   Installed DataFrames ───────────── v0.20.2
   Installed BinDeps ──────────────── v1.0.2
   Installed XZ_jll ───────────────── v5.4.5+0
   Installed fzf_jll ──────────────── v0.43.0+0
   Installed Missings ─────────────── v0.4.5
   Installed TrafficAssignment ────── v0.6.3
   Installed NaNMath ──────────────── v0.3.7
   Installed BenchmarkTools ───────── v1.4.0
   Installed NLSolversBase ────────── v7.5.0
   Installed libpng_jll ───────────── v1.6.40+0
   Installed Compat ───────────────── 

-------------------------------------
Network Name: Problem1
Method: cfw
Line Search Step: newton
Maximum Interation Number: 1000
Tolerance for AEC: 1.0e-5
Number of processors: 1
Setup time = 0.09076380729675293 seconds
k=   1, tauk=            NaN, objective=   63675.000000, aec=   0.0000000000
Iteration time = 0.2001039981842041 seconds
Link flow: [100.0, 60.0, 40.0, 0.0]
Link travel time: [10.0, 635.0, 2125.0, 125.0]
Objective: 63675.0
System travel time: 124100.0
